# A Simple Cryptocurrency Program

In this workbook we mock up a bitcoin like cryptocurrency. 

Some knowledge of Python *classes* is needed, which will refresh the reader about object-oriented programming. 

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Wallet

A **Wallet** is simply a data object comprising a public and private key. 
- The public key is used to receive currency.
- The private key is used to sign a transaction when sending money. The public key can be used by anyone to verify the transaction. 

**RSA in one sentence**: If $A$ (Alice) is sending money to $B$ (Bala) via transaction $T$, then Alice encodes the transaction with Bala's public key, and further encodes the result with her private key. Bala can then decode the transaction using Alice's public key, and further uncover transaction $T$ using his private key. 

So, Alice sends the following to Bala:

$$
a = PrivKey_A(PubKey_B(T))
$$

And Bala decodes the transaction through the application of two corresponding functions:

$$
T = PrivKey_B(PubKey_A(a))
$$

In [2]:
import Crypto
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5

import binascii

In [3]:
# We will be using a decorator later
# An aside: Decorators extend functions, here is an example.
# decorators: https://realpython.com/blog/python/primer-on-python-decorators/

import time

def timing_function(some_function):
    def wrapper():
        t1 = time.time()
        some_function()
        t2 = time.time()
        return "Time it took to run the function: " + str((t2 - t1)) + "\n"
    return wrapper

@timing_function
def my_function():
    num_list = []
    for num in (range(0, 10000)):
        num_list.append(num)
    print("Sum of all the numbers: " + str(sum(num_list)))

print(my_function())  #Same as calling timing_function(my_function)

Sum of all the numbers: 49995000
Time it took to run the function: 0.0027840137481689453



## Create Wallet class

In which we also initiate new programmers to classes and OOP, decorators, and assertions. 

In [4]:
class Wallet(object):
    
    def __init__(self):
        x = Crypto.Random.new().read   #initialize wallet with random number for keys
        self.PrivKey = RSA.generate(1024,x)  
        self.PubKey = self.PrivKey.publickey()
        self.signer = PKCS1_v1_5.new(self.PrivKey)  
        #RSA digital signature protocol according to PKCS#1 v1.5
        
    # Define the wallet address as its public key to keep things simple
    @property   
    def address(self):
        return binascii.hexlify(self.PubKey.exportKey(format='DER')).decode('ascii')
        #DER: Distinguished Encoding Rules
    
    #Hash the message and then sign it with the private key
    def sign(self, message):
        m = SHA.new(message.encode('utf8'))
        return binascii.hexlify(self.signer.sign(m)).decode('ascii')

In [5]:
#Examples
x = Crypto.Random.new().read 
priv = RSA.generate(1024,x)
pub = priv.publickey()
pub1 = binascii.hexlify(pub.exportKey(format='DER'))
print(pub1)
print('\n')
print(binascii.hexlify(pub1))   #Every ascii character gets the 2 character hex representation

b'30819f300d06092a864886f70d010101050003818d0030818902818100b8c254e86af3ae0e1e7d02cb07a881dfd0e74a49c4b1154522321da8f3138a2a690486a957a07eb431eb7c7ffbb6fe0be990517cfe91120808cd4a7be6c2ab4636addcb3ca84c9cdd65be67f3c2226353d920adecaeb1463765048e263db1b8e5e506d52e2f3b15707080a91d7e89fe0c5a38dd2f1055c1a156464bad45c64f90203010001'


b'333038313966333030643036303932613836343838366637306430313031303130353030303338313864303033303831383930323831383130306238633235346538366166336165306531653764303263623037613838316466643065373461343963346231313534353232333231646138663331333861326136393034383661393537613037656234333165623763376666626236666530626539393035313763666539313132303830386364346137626536633261623436333661646463623363613834633963646436356265363766336332323236333533643932306164656361656231343633373635303438653236336462316238653565353036643532653266336231353730373038306139316437653839666530633561333864643266313035356331613135363436346261643435633634663930323033303130303031'


## Wallet signing 

In [6]:
def verify_signature(wallet_address, message, signature):
    pubkey = RSA.importKey(binascii.unhexlify(wallet_address))
    verifier = PKCS1_v1_5.new(pubkey)
    m = SHA.new(message.encode('utf8'))
    return verifier.verify(m, binascii.unhexlify(signature))

Check that verification works. 
- Initialize the wallet. This creates within the wallet a private key, public key = address, signer based on private key. The wallet has functions for generating the address and for signing a message.
- Sign a message from the wallet using the 'sign' function. 
- Call the 'verify_signature' function by the wallet address, pass the message, and the signature for verification. 

In [7]:
w1 = Wallet()
signature = w1.sign('valid message')
print(signature)

#assert throws an exception only if the asserted statement is false.
assert verify_signature(w1.address, 'valid message', signature)
assert not verify_signature(w1.address, 'false message', signature)

print('\n'); 
print(verify_signature(w1.address, 'valid message', signature))
print(verify_signature(w1.address, 'false message', signature))

150da4be702a9faa179ecec390d900c4266b061c5c46655c3a0c342b441e1600379ff86fd7a9b7c58031120e7c358f64ecfac92c4f2bbfc86b8742ddca83294966823d928c77cbbe7a370935e31ccb2cdd87f1c6c8b776fe52b65a4e2e5c9d0b30e429e36e7f4eb4531be569e4db43c73568065fe8c51ffd34678cd9cd2c95bb


True
False


## Hashing and Proof of Work (PoW)

In which we explore SHA-256 hashing to see how it works, hexlify and unhexlify, as already seen above. 

In [8]:
import hashlib
import random
import string
import json

- Take an ascii message, encode it, hash it, and hex it. The hex gives a 64 character hexadecimal representation of the hashed message. 

- Mining requires taking the message, adding a nonce (number) and then hashing it to check if a given number of leading zeros is achieved. 

In [9]:
def sha256(message):
    return hashlib.sha256(message.encode('ascii')).hexdigest()

In [10]:
m = "My test message"
for nonce in range(10000):
    h = sha256(m + str(nonce))
    if h.startswith('00'):
        print(nonce,h)
        break

207 000bd808dc79f4cf5dc658fcc3364346b35809414e07f51fbee14af486155654


For different levels of **difficulty** we can see how long it takes to solve the PoW.

In [11]:
for difficulty in range(1,6):
    for nonce in range(1000000):
        h = sha256(m + str(nonce))
        d = '0' * difficulty
        if h.startswith(d):
            print(difficulty,nonce,h)
            break

1 84 02b797d5eb4a6b296cbf7045569b51f19a771f163940fee815d993b541124679
2 207 000bd808dc79f4cf5dc658fcc3364346b35809414e07f51fbee14af486155654
3 207 000bd808dc79f4cf5dc658fcc3364346b35809414e07f51fbee14af486155654
4 114361 0000968550219036cfaf362e180217e2c17e40f53ae3a767fba8636d519abe49
5 690875 0000096817366c594b2e93163d45f7aa7043b22f6ad48d08c5e25c7b0f03a38a


## Transactions

In which we see how a transaction will be performed in code. 

Each transaction will have an input, held by one person, which is then given to a receiver partially in terms of payment, the remainder less fees being paid back the sender. 

The Transaction Input points to the output of another transaction. It is the amount of coin from another previous transaction that came in. Transaction Outputs will be amounts send to (i) Receiver, (ii) remaining to Sender, (iii) fees. 

In [12]:
class TransactionInput(object):

    def __init__(self, transaction, output_index):
        self.transaction = transaction
        self.output_index = output_index
        assert 0 <= self.output_index < len(transaction.outputs)
    
    def to_dict(self):
        d = {
            'transaction': self.transaction.hash(),
            'output_index': self.output_index
            }
        return d

    @property
    def parent_output(self):
        return self.transaction.outputs[self.output_index]

The Transaction Ouput relates to a Transaction Input. It states the Recepient and the Amount. 

In [13]:
class TransactionOutput(object):

    def __init__(self, recipient_address, amount):
        self.recipient = recipient_address
        self.amount = amount

    def to_dict(self):
        d = {
            'recipient_address': self.recipient,
            'amount': self.amount
        }
        return d

In [14]:
def compute_fee(inputs, outputs):
    total_in = sum(i.transaction.outputs[i.output_index].amount for i in inputs)
    total_out = sum(o.amount for o in outputs)
    assert total_out <= total_in, "Invalid transaction with out(%f) > i(%f)" % (total_out, total_in)
    return total_in - total_out

In [15]:
class Transaction(object):
    
    def __init__(self, wallet, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs
        self.fee = compute_fee(inputs, outputs)
        self.signature = wallet.sign(json.dumps(self.to_dict(include_signature=False)))

    def to_dict(self, include_signature=True):
        d = {
            "inputs": list(map(TransactionInput.to_dict, self.inputs)),
            "outputs": list(map(TransactionOutput.to_dict, self.outputs)),
            "fee": self.fee
            }
        if include_signature:
            d["signature"] = self.signature
            return d
        
    def hash(self):
        return sha256(json.dumps(self.to_dict()))

*Genesis transaction*: This is the first transaction which is a special transaction with no input and 25 bitcoins output.

In [16]:
class GenesisTransaction(Transaction):

    def __init__(self, recipient_address, amount=50.0):
        self.inputs = []
        self.outputs = [TransactionOutput(recipient_address, amount)]
        self.fee = 0
        self.signature = 'genesis'

    def to_dict(self, include_signature=False):
        assert not include_signature, "Cannot include signature of genesis transaction"
        return super().to_dict(include_signature=False)

And now, we can finally execute a transaction. Two wallets, initialize the first genesis transaction, where Alice gets 50 coin, and then Alice does transaction 2, giving 10 coin to Bala, with remaining 39 coin, after paying a fee of 1. The fee is the difference between the inputs and the outputs. 

In [17]:
alice = Wallet()
bala = Wallet()
t1 = GenesisTransaction(alice.address)
t2 = Transaction(alice,[TransactionInput(t1, 0)],[TransactionOutput(bala.address, 10.0), 
                                                  TransactionOutput(alice.address, 39.0)])
assert np.abs(t2.fee - 1.0) < 1e-5

Let's take a few more transactions.

In [18]:
chandra = Wallet()
t3 = Transaction(bala,[TransactionInput(t2,0)],[TransactionOutput(chandra.address, 7.0),
                                                TransactionOutput(bala.address,3.0)])
t4 = Transaction(chandra,[TransactionInput(t3,0)],[TransactionOutput(alice.address, 5.0),
                                                TransactionOutput(chandra.address,1.0)])

## Balances

Where we show how to run through transactions in order to generate current balances of coin. 

In [19]:
transactions = [t1,t2,t3,t4]

In [20]:
def compute_balance(wallet_address, transactions):
    balance = 0.0
    for t in transactions:
        # Subtract all the money that the address sent out
        for txin in t.inputs:
            if txin.parent_output.recipient == wallet_address:
                balance = balance - txin.parent_output.amount
        # Add all the money received by the address
        for txout in t.outputs:
            if txout.recipient == wallet_address:
                balance = balance + txout.amount
    return balance

In [21]:
compute_balance(alice.address,transactions)

44.0

In [22]:
compute_balance(bala.address,transactions)

3.0

In [23]:
compute_balance(chandra.address,transactions)

1.0

These balances are also known as UTXOs, or **Unspent Transaction Outputs**. 

Note that 2 coins were paid out in fees and are in some miner's wallet. 

We also note that before the transaction itself can come into existence, we need to make sure that it is verified through PoW. So all these transactions would only be executed by the functions after they were put into a single block, hashed, and then a nonce was found by some miner. 